In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
from datetime import date
import sqlite3

In [ ]:
# carregando arquivo json
df = pd.read_json('../data/data.json')
df.head(5)

In [ ]:
# criação das colunas que indicam a fonte dos dados e data da coleta
df['fonte'] = 'https://lista.mercadolivre.com.br/tenis-corrida-masculino'
df['data_coleta'] = date.fromisoformat('2024-05-24')
df.head()

In [ ]:
# remover parentese da coluna qnt_avaliacoes e converter a coluna para o tipo int
df['qnt_avaliacoes'] = (
    df['qnt_avaliacoes']
    .str.replace(r"[\(\)]", '', regex=True)
)
df['qnt_avaliacoes'] = (
    pd.to_numeric(df['qnt_avaliacoes'], errors='coerce')
    .fillna(0)
    .astype(int)
)

df[['qnt_avaliacoes']]

In [ ]:
df.dtypes

In [ ]:
# tratando preços e calculando valores totais
df['preco_antigo'] = df['valor_antigo'] + df['centavos_ant'] / 100
df['novo_preco'] = df['valor_novo'] + df['centavos_nv'] / 100

In [ ]:
# removendo colunas de preço antigas
df.drop(columns=['valor_antigo','centavos_ant','valor_novo','centavos_nv'])

In [ ]:
# tratandondo valores nulos
df.fillna(0).head(5)

In [ ]:
# conectando ao banco de dados
conn = sqlite3.connect('../data/cotas.db')

In [ ]:
# salvando o dataframe no banco de dados
df.to_sql('mercadolivre_items', conn, if_exists='replace', index=False)

In [ ]:
conn.close()